# **Hands-on Lab: Interactive Visual Analytics with Folium**

This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

In [6]:
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()#.first()返回各組第一個非空值
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
launch_sites = [
    {"name": "CCAFS LC-40", "coordinate": [28.562302, -80.577356]},
    {"name": "CCAFS SLC-40", "coordinate": [28.563197, -80.576820]},
    {"name": "KSC LC-39A", "coordinate": [28.573255, -80.646895]},
    {"name": "VAFB SLC-4E", "coordinate": [34.632834, -120.610745]},
]
# 添加圓形和標記
for site in launch_sites:
    # 添加圓形標記
    circle = folium.Circle(
        location=site["coordinate"],
        radius=10000,
        color="blue",
        fill=True,
    ).add_child(folium.Popup(site["name"]))
    
    # 添加標記與名稱
    marker = folium.map.Marker(
        location=site["coordinate"],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site["name"]}</b></div>',
        ),
    )
    
    # 將標記與圓形添加到地圖
    site_map.add_child(circle)
    site_map.add_child(marker)

# 顯示地圖
site_map

An example of folium.Circle:
`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`

An example of folium.Marker:
`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
# Task 2: Mark the success/failed launches for each site on the map
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [11]:
marker_cluster = MarkerCluster()

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [16]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    if record['class']==1:
        color='green'
        word='success'
    elif record['class']==0:
        color='red'
        word='failure'
    coordinate=[record['Lat'],record['Long']]  
    marker=folium.Marker(location=coordinate,
                         icon=folium.Icon(color='white',icon_color=color),
                         popup=word)                                 
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [14]:
#TASK 3: Calculate the distances between a launch site to its proximities

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [18]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [20]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
airport_pos=[28.41,-80.30]
airport_distance=calculate_distance(28.562302,-80.577356,28.41,-80.30)

In [21]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
distance_marker=folium.Marker(airport_pos,
                              icon=DivIcon(
                                  icon_size=(20,20),
                                  icon_anchor=(0,0),
                                  html='<div style="font-size: 12; color:red;"><b>%s</b></div>' % "{:10.2f} KM".format(airport_distance),
                              ))
distance_marker.add_to(site_map)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point

In [22]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
coordinates=[[28.562302,-80.577356],airport_pos]
lines=folium.PolyLine(locations=coordinates,weight=1)
site_map.add_child(lines)

*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


In [23]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
railway=[28.44329,-80.77]
city=[28.81,-81.26]
railway_distance=calculate_distance(28.562302,-80.577356,28.44329,-80.77)
city_distance=calculate_distance(28.562302,-80.577356,28.81,-81.26)

In [24]:
railway=[28.44329,-80.77]
city=[28.81,-81.26]
railway_distance=calculate_distance(28.562302,-80.577356,28.44329,-80.77)
city_distance=calculate_distance(28.562302,-80.577356,28.81,-81.26)
coordinate1=[[28.562302,-80.577356],city]
coordinate2=[[28.562302,-80.577356],railway]
distance_city_marker=folium.Marker(city,
                              icon=DivIcon(
                                  icon_size=(20,20),
                                  icon_anchor=(0,0),
                                  html='<div style="font-size: 12; color:red;"><b>%s</b></div>' % "{:10.2f} KM".format(city_distance),
                              ))
distance_railway_marker=folium.Marker(railway,
                              icon=DivIcon(
                                  icon_size=(20,20),
                                  icon_anchor=(0,0),
                                  html='<div style="font-size: 12; color:red;"><b>%s</b></div>' % "{:10.2f} KM".format(railway_distance),
                              ))
lines1=folium.PolyLine(locations=coordinate1,weight=1)
site_map.add_child(lines1)
lines2=folium.PolyLine(locations=coordinate2,weight=1)
site_map.add_child(lines2)
distance_city_marker.add_to(site_map)
distance_railway_marker.add_to(site_map)
site_map